# Earthquake seismology with DAS

#### Todo List:
1. Reference earthquakes by Evid rather than dataframe index
2. Make a map of all of the detected events
3. Try some xcorr

In [1]:
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import numpy as np
import datetime
import pandas as pd
from dasquakes import sintela_to_datetime, open_sintela_file, local_earthquake_quicklook, data_wrangler, haversine
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from libcomcat.search import search
from libcomcat.dataframes import get_summary_data_frame

ERROR 1: PROJ: proj_create_from_database: Open of /home/velgueta/.conda/envs/python-obspy/share/proj failed


### Get a list of events from USGS

In [2]:
events = search(starttime = datetime.datetime(2022, 3, 17, 0, 0), 
                endtime   = datetime.datetime.now(),
                minmagnitude=3.0,
                maxradiuskm=500, 
                latitude=47.60, 
                longitude=-122.33)

print("Returned %s events" % len(events))
 

Returned 16 events


### Choose an event to analyze

In [3]:
event_df = get_summary_data_frame(events)
event_df = event_df.sort_values(by=['time'],ascending=False)
event_df


#event_df.iloc[0::]

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
15,us6000jk08,2023-01-28 16:10:51.411,None,48.999000,-128.559200,10.000,4.00,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,247
14,us7000j3g5,2022-12-28 08:07:48.135,off the coast of Washington,47.841000,-128.912500,11.797,3.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,234
13,us7000j3f8,2022-12-28 08:06:54.980,"267 km SW of Tofino, Canada",47.804700,-128.908400,9.158,3.80,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,222
12,us7000ishe,2022-11-26 03:50:17.526,"Vancouver Island, Canada region",49.271400,-126.092300,33.380,4.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,527
11,uw61886506,2022-10-07 12:52:36.010,"16 km ESE of Lacomb, Oregon",44.540333,-122.550833,13.170,4.39,green,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,896
10,us6000ipd3,2022-09-29 15:09:32.565,off the coast of Washington,47.637400,-128.921800,10.555,3.30,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,168
9,uw61882431,2022-09-22 14:41:56.900,"26 km WNW of Forks, Washington",48.046500,-124.715000,37.040,3.49,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,190
8,uw61869652,2022-09-08 08:08:25.830,"7 km WNW of Lofall, Washington",47.842000,-122.751667,23.330,3.71,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,592
7,uw61862977,2022-08-13 13:31:22.900,"21 km NW of Bickleton, Washington",46.130333,-120.500000,12.100,3.05,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,146
6,us6000iac9,2022-08-12 20:00:04.319,"148 km W of Yachats, Oregon",44.483100,-125.947500,10.000,4.00,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,246


In [4]:
for index,this_event in event_df.iterrows():
    
    this_id = this_event['id']
    
    this_event_new = event_df[event_df.id==this_id]
    
    latitude_loc = this_event_new['latitude']
    longitude_loc = this_event_new['longitude']
    latitude_ref = 47.60
    longitude_ref = -122.33
    
    distance = haversine(longitude_loc,latitude_loc,longitude_ref,latitude_ref)
    
    #if distance < 500:
    #    continue
    
    print(this_event['id'])
    
    stitle=f"M {this_event_new['magnitude'].to_numpy()[0]}, "\
           f"{this_event_new['location'].to_numpy()[0]}, "\
           f"{str(this_event_new['time'].to_numpy()[0])[0:-10]} UTC"
      

        
    cable = 'whidbey'
    record_length = 10 #minutes
    t0 = this_event_new['time'].iloc[0]
    
    prefix, network_name, datastore = data_wrangler(cable,record_length,t0)
        
    try:
        data,dates,attrs = open_sintela_file(prefix,
                                             t0,
                                             datastore,
                                             number_of_files=record_length,
                                             verbose=True)
        x_max=data.shape[1] * attrs['SpatialSamplingInterval']
        
        low_cut = 2
        hi_cut = 10

        b,a = butter(2,(low_cut,hi_cut),'bp',fs=attrs['MaximumFrequency']*2)
        data_filt = filtfilt(b,a,data,axis=0)
        
        filename = f"Earthquakes500betterchannels/"+this_id
        local_earthquake_quicklook(dates,20*data_filt,0,0,x_max,
                           stitle,filename=filename,das_vmax=0.02,
                           network_name=network_name,
                           skip_seismograms=True)
       # timestamps = []
        #for d in dates:
         #   if type(d) == datetime.datetime:
         #       timestamps.append(int((d.timestamp()-25200)*10**6))
          #  else:
           #     timestamps.append(d)
        #g=h5py.File(filename+'.h5','w')
        #g.create_dataset('datachannel',data=data_filt)
        #g.create_dataset('time',data=np.array(timestamps))
        #g.attrs.update(attrs)
       
        #g.close()
        
    except ValueError:
        continue
        # x_max=data.shape[1] * attrs['SpatialSamplingInterval']
        
   # low_cut = 2
    #hi_cut = 10

    #b,a = butter(2,(low_cut,hi_cut),'bp',fs=attrs['MaximumFrequency']*2)
    #data_filt = filtfilt(b,a,data,axis=0)
        
    #filename = f"earthquakes_figures"
    #local_earthquake_quicklook(dates,data_filt,x_max,
                      #     stitle,filename=filename,das_vmax=0.02,
                       #    network_name=network_name,
                        #   skip_seismograms=True)


us6000jk08
Searching for files matching: /data/data6/whidbey/whidbey_2023-01-28_16-10*h5
us7000j3g5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-07*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-08*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-09*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-10*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-11*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-12*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-13*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-14*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-15*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-16*h5
us7000j3f8
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-06*h5
Searching for files matching: /

Searching for files matching: /data/data5/Converted/whidbey_2022-08-04_13-23*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-04_13-24*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-04_13-25*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-04_13-26*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-04_13-27*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-04_13-28*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-04_13-29*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-04_13-30*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-04_13-31*h5
uw61836427
Searching for files matching: /data/data5/Converted/whidbey_2022-05-28_19-07*h5
us6000hn38
Searching for files matching: /data/data5/Converted/whidbey_2022-05-22_09-58*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-05-22_09-59*h5
Searching for file